In [1]:
import requests
import json
import numpy as np
import pandas as pd
import math
from bs4 import BeautifulSoup

In [2]:
def getData(page,typeD):
    url = "https://xapi.supercharge-srp.co/job-search/graphql?country=my&isSmartSearch=true"
    payload = json.dumps({
        "query": "query getJobs($country: String, $locale: String, $keyword: String, $createdAt: String, $jobFunctions: [Int], $categories: [String], $locations: [Int], $careerLevels: [Int], $minSalary: Int, $maxSalary: Int, $salaryType: Int, $candidateSalary: Int, $candidateSalaryCurrency: String, $datePosted: Int, $jobTypes: [Int], $workTypes: [String], $industries: [Int], $page: Int, $pageSize: Int, $companyId: String, $advertiserId: String, $userAgent: String, $accNums: Int, $subAccount: Int, $minEdu: Int, $maxEdu: Int, $edus: [Int], $minExp: Int, $maxExp: Int, $seo: String, $searchFields: String, $candidateId: ID, $isDesktop: Boolean, $isCompanySearch: Boolean, $sort: String, $sVi: String, $duplicates: String, $flight: String, $solVisitorId: String) {\n  jobs(\n    country: $country\n    locale: $locale\n    keyword: $keyword\n    createdAt: $createdAt\n    jobFunctions: $jobFunctions\n    categories: $categories\n    locations: $locations\n    careerLevels: $careerLevels\n    minSalary: $minSalary\n    maxSalary: $maxSalary\n    salaryType: $salaryType\n    candidateSalary: $candidateSalary\n    candidateSalaryCurrency: $candidateSalaryCurrency\n    datePosted: $datePosted\n    jobTypes: $jobTypes\n    workTypes: $workTypes\n    industries: $industries\n    page: $page\n    pageSize: $pageSize\n    companyId: $companyId\n    advertiserId: $advertiserId\n    userAgent: $userAgent\n    accNums: $accNums\n    subAccount: $subAccount\n    minEdu: $minEdu\n    edus: $edus\n    maxEdu: $maxEdu\n    minExp: $minExp\n    maxExp: $maxExp\n    seo: $seo\n    searchFields: $searchFields\n    candidateId: $candidateId\n    isDesktop: $isDesktop\n    isCompanySearch: $isCompanySearch\n    sort: $sort\n    sVi: $sVi\n    duplicates: $duplicates\n    flight: $flight\n    solVisitorId: $solVisitorId\n  ) {\n    total\n    totalJobs\n    relatedSearchKeywords {\n      keywords\n      type\n      totalJobs\n    }\n    solMetadata\n    suggestedEmployer {\n      name\n      totalJobs\n    }\n    queryParameters {\n      key\n      searchFields\n      pageSize\n    }\n    experiments {\n      flight\n    }\n    jobs {\n      id\n      adType\n      sourceCountryCode\n      isStandout\n      companyMeta {\n        id\n        advertiserId\n        isPrivate\n        name\n        logoUrl\n        slug\n      }\n      jobTitle\n      jobUrl\n      jobTitleSlug\n      description\n      employmentTypes {\n        code\n        name\n      }\n      sellingPoints\n      locations {\n        code\n        name\n        slug\n        children {\n          code\n          name\n          slug\n        }\n      }\n      categories {\n        code\n        name\n        children {\n          code\n          name\n        }\n      }\n      postingDuration\n      postedAt\n      salaryRange {\n        currency\n        max\n        min\n        period\n        term\n      }\n      salaryVisible\n      bannerUrl\n      isClassified\n      solMetadata\n    }\n  }\n}\n",
        "variables": {
            "keyword": "data analyst",
            "jobFunctions": [],
            "locations": [
                70000
            ],
            "salaryType": 1,
            "jobTypes": [],
            "createdAt": "1d",
            "careerLevels": [],
            "page": page,
            "country": "sg",
            "sVi": "[CS]v1|318D4B8CC4E76ACA-600012A7D3E18519[CE]",
            "categories": [],
            "workTypes": [
                "full_time"
            ],
            "userAgent": "Mozilla/5.0%20(Windows%20NT%2010.0;%20Win64;%20x64)%20AppleWebKit/537.36%20(KHTML,%20like%20Gecko)%20Chrome/109.0.0.0%20Safari/537.36",
            "industries": [],
            "locale": "en"
        }
    })
    headers = {
        'accept': '*/*',
        'content-type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    if typeD == "total":
        return response.json()['data']['jobs']["total"]
    elif typeD == "jobs":
        return response.json()['data']['jobs']["jobs"]

In [3]:
def maxPages(data):
    return math.ceil(data/30)

In [4]:
totalPages = maxPages(getData(1,"total"))

In [5]:
results=[]

for i in range(totalPages):
    data = getData(i,"jobs")
    for job in data:      
        results.append([
            job['jobTitle'], job['description'], job['jobUrl'],job['companyMeta']['name'], job['salaryRange']["max"],job['postedAt']
        ])
df_full = pd.DataFrame(results)
df_full.columns = ['Title', 'Description', 'Link', 'Company Name','Max Salary','Post Date']

In [6]:
df_full['Title']=df_full['Title'].apply(lambda x: x.lower())

In [7]:
title_filter= ["manager","senior",'leader','head','mgr','director','lead','intern','contract','credit','research','executive','scientist']
title_filter2=['graduate','analyst','intelligence','insight', 'data']

df_f_title=df_full[df_full["Title"].str.contains("|".join(title_filter))==False]
df_f_title=df_f_title[df_f_title["Title"].str.contains("|".join(title_filter2))==True]
len(df_f_title)

420

In [8]:
def getHTML(url):
    response = requests.request("GET", url)
    html_soup= BeautifulSoup(response.text, 'html.parser')
    return html_soup

def getJD(html_soup):
    try:
        jobDescription = html_soup.find_all("div", {"class": "_1x1c7ng0"})[0].get_text()
    except:
        jobDescription = "not found"
    return jobDescription

def getApplyDest(html_soup):
    try:
        applyLink = html_soup.find_all("a", {"data-automation": "applyNowButton"})[0]['href']
    except:
        return "Cant Find Apply Link"
    
    if 'https://sg.jobsdb.com' not in applyLink:
        return "Not CareersFuture"
    
    response = requests.request("GET", applyLink)
    html_soup= BeautifulSoup(response.text, 'html.parser')
    
    try:
        result =html_soup.find_all("script")[0].text
    except:
        return "Cant Find Script"
    if 'https://www.mycareersfuture.gov.sg' not in result:
        return "Not CareersFuture"
    else:
        return "CareersFuture"

def HTMLdata(url):
    soup=getHTML(url)
    return [getApplyDest(soup), getJD(soup)]

def expandData(x):
    data = HTMLdata(x[2])
    x['CF?'] =data[0]
    x['JD'] = data[1]
    return x

In [9]:
df_f_title=df_f_title.sort_values(by=['Title'])
df_f_title=df_f_title.drop_duplicates(subset=['Title','Description', 'Company Name'], keep='last')
print(len(df_f_title))
df_f_title.head()

403


,Title,Description,Link,Company Name,Max Salary,Post Date
594,"*perm* analyst ($3,800 to $4,500) #nja",The Requirements: Work closely with stakeholde...,https://www.jobstreet.com.my/en/job/*perm*-ana...,Recruit Express Pte Ltd,NaN,2023-03-04T17:47:49Z
2652,2023 commodities and global markets – quant st...,2023 Commodities and Global Markets – Quant St...,https://www.jobstreet.com.my/en/job/2023-commo...,Macquarie (Asia) Pte Ltd,NaN,2023-03-04T17:45:52Z
726,2023 wealth management full-time analyst progr...,DescriptionOur Wealth Management Analyst Progr...,https://www.jobstreet.com.my/en/job/2023-wealt...,Credit Suisse (Singapore) Limited,NaN,2023-03-04T20:42:24Z
373,22600225 data analyst,The Data Governance Senior Analyst is a season...,https://www.jobstreet.com.my/en/job/22600225-d...,Citibank N.A.,NaN,2023-03-04T20:42:24Z
614,23611971 strategy & rewards management analyst...,The Strategy and Rewards Management Analyst ac...,https://www.jobstreet.com.my/en/job/23611971-s...,Citibank N.A.,NaN,2023-03-04T20:42:39Z


In [10]:
df_f_title = df_f_title.apply(expandData, axis=1)
df_f_title.head()

,Title,Description,Link,Company Name,Max Salary,Post Date,CF?,JD
594,"*perm* analyst ($3,800 to $4,500) #nja",The Requirements: Work closely with stakeholde...,https://www.jobstreet.com.my/en/job/*perm*-ana...,Recruit Express Pte Ltd,NaN,2023-03-04T17:47:49Z,CareersFuture,The Requirements:Work closely with stakeholder...
2652,2023 commodities and global markets – quant st...,2023 Commodities and Global Markets – Quant St...,https://www.jobstreet.com.my/en/job/2023-commo...,Macquarie (Asia) Pte Ltd,NaN,2023-03-04T17:45:52Z,Not CareersFuture,2023 Commodities and Global Markets – Quant St...
726,2023 wealth management full-time analyst progr...,DescriptionOur Wealth Management Analyst Progr...,https://www.jobstreet.com.my/en/job/2023-wealt...,Credit Suisse (Singapore) Limited,NaN,2023-03-04T20:42:24Z,CareersFuture,DescriptionOur Wealth Management Analyst Progr...
373,22600225 data analyst,The Data Governance Senior Analyst is a season...,https://www.jobstreet.com.my/en/job/22600225-d...,Citibank N.A.,NaN,2023-03-04T20:42:24Z,CareersFuture,The Data Governance Senior Analyst is a season...
614,23611971 strategy & rewards management analyst...,The Strategy and Rewards Management Analyst ac...,https://www.jobstreet.com.my/en/job/23611971-s...,Citibank N.A.,NaN,2023-03-04T20:42:39Z,CareersFuture,The Strategy and Rewards Management Analyst ac...


In [11]:
df_f_title_CF=df_f_title[df_f_title["CF?"]!="CareersFuture"]

df_f_title_CF['Link']=df_f_title_CF['Link'].str.replace('https://www.jobstreet.com.my', 'https://www.jobstreet.com.sg')
df_f_title_CF=df_f_title_CF.reset_index(drop=True)
print(len(df_f_title_CF))

89


c:\users\liyif\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 200)
df_f_title_CF

,Title,Description,Link,Company Name,Max Salary,Post Date,CF?,JD
0,2023 commodities and global markets – quant strategy graduate role,2023 Commodities and Global Markets – Quant Strategy Graduate RoleSingaporeThis is an exciting opportunity for graduates to join our Quant Strategy...,https://www.jobstreet.com.sg/en/job/2023-commodities-and-global-markets-quant-strategy-graduate-role-1034685828/origin/sg?token=0~41fd3608-5ce0-4ecc-b889-7b2a10dd4d6e&sectionRank=2623&jobId=jobstreet-sg-job-1034685828,Macquarie (Asia) Pte Ltd,NaN,2023-03-04T17:45:52Z,Not CareersFuture,"2023 Commodities and Global Markets – Quant Strategy Graduate RoleSingaporeThis is an exciting opportunity for graduates to join our Quant Strategy team. This is a front-office role based in Singapore and supporting our commodity and global markets businesses.Our Quant Strategy team applies specialist methods from mathematics, science, and engineering to generate revenue. We work on data, models, and algorithms with application to derivative valuation and risk, automated trading, and data-driven decision-making.Your responsibilities will be to; help research and implement new structured products, provide quantitative expertise to traders, and work across divisions of the bank to extend and evolve our quantitative capabilities. You’ll be part of a highly respected and supportive team and will be given responsibility and the opportunity to learn and develop from an early stage.If you have the aptitude for adapting to a dynamic front office environment, the skills to work with variou..."
1,actuarial analyst (m/f/d),"What you doYou will be working as the Corporate Actuarial, part of the Finance division, and your function will be to operate globally with a direct...",https://www.jobstreet.com.sg/en/job/actuarial-analyst-m-f-d-1034760245/origin/sg?token=0~674570be-44ea-48e7-83b1-7f3e16908fb6&sectionRank=532&jobId=jobstreet-sg-job-1034760245,Allianz,NaN,2023-03-04T17:05:47Z,Not CareersFuture,"What you doYou will be working as the Corporate Actuarial, part of the Finance division, and your function will be to operate globally with a direct relevance to the business, following the practices of Technical Excellence in the Allianz Group. This role is based in the Singapore office, and reports to the Head of Corporate Actuarial Asia Pacific.You will be coordinating with the local businesses across Asia Pacific and the other Corporate Actuarial locations, you will support the implementation of the reserving strategy for the Travel, Health and Assistance lines of business. This includes the reserve setting, assessment of underwriting performance, support to Solvency II actuarial function activities and contributing to other actuarial and Finance projects.In order to optimize the company’s financial performance, you will challenge existing processes, analyse deviations, proactively propose corrective actions, elaborate mitigation plans and monitor implementation.You would be ex..."
2,analyst,Job DescriptionThe data management and public market support team in Singapore are looking for new colleagues. The role would consist of the...,https://www.jobstreet.com.sg/en/job/analyst-1034683211/origin/sg?token=0~43951440-a8c0-42e4-b735-1297a35e25d1&sectionRank=785&jobId=jobstreet-sg-job-1034683211,Standard Life Aberdeen,NaN,2023-03-04T17:38:29Z,Not CareersFuture,"Job DescriptionThe data management and public market support team in Singapore are looking for new colleagues. The role would consist of the following:Responsibilities include:Data Management APACSet up and maintenance of Portfolio DataDaily checks and corrections on regional data received from third party administrators e.g. positions, security, prices, analyticsMaintenance of security and static data on Charles River and other applicationsProvide 1st line support for regional data issues – query logging and investigationLiaison with internal teams including, key stakeholders, global data team and regional teams in the US and E

In [13]:
#test.to_csv('jobstreet_3d_analyst_jobs_14022023.csv',index=False)